![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<h1 align='center'>Labour force characteristics by Indigenous group and educational attainment</h1>

## How to run this notebook:

Press Kernel, Restart and Run all Cells, Confirm Restart and Run all cells. 

Alternatively, run each cell using the Run button. 

## About this notebook

In this notebook, we explore labour force characteristics, where the population is broken down into the following categories:

1. Indigenous group
    
        Total population
        Indigenous population
        First Nations
        Métis
        Non-Indigenous population

2. Age group

        15 years and over
        25 years and over
        25 to 54 years

3. Labour force characteristics

        Population
        Labour force
        Employment
        Unemployment
        Not in labour force
        Unemployment rate
        Participation rate
        Employment rate
        
        
4. Educational attainment

        Total, all education levels
        Less than high school
        High school graduate or some post-secondary
        Completed post-secondary educatio

[Citation] 
Statistics Canada.  Table  14-10-0359-01   Labour force characteristics by Indigenous group and educational attainment
DOI:   https://doi.org/10.25318/1410035901-eng


## What are the questions we are interested in answering? 

1. What are the differences in highest educational level by age group? 

2. What are the differences in labour force status, by age group and Indigenous, Metis or First Nations status? 

3. How have employment and unemployment rates have changed since 2007 up until today?

4. How can you use the data in this notebook to help you identify areas of improvement? 


In [ ]:
%run -i ./StatsCan/helpers.py
%run -i ./StatsCan/scwds.py
%run -i ./StatsCan/sc.py
%run -i ./widgets_libraries.py

## Downloading Stats Can Data

To download a full dataset, run the cell below and press on the "Download Dataset" button.

In [ ]:
prod_ID = widgets.Text(
    value="14-10-0359-01",
    placeholder='ProductID value',
    description='productID value',
    disabled=False,
    style=style
)


DS_button = widgets.Button(
    button_style='success',
    description="Download Dataset", 
    layout=Layout(width='15%', height='30px'),
    style=style
)    
DS_button.on_click( run_4cell )
display(prod_ID)
display(DS_button)

In [ ]:
# # Download data 
productId = prod_ID.value

if "-" not in productId:
    if len(productId)!=10:
        print("WARNING: THIS IS LIKELY A NUMBER NOT ASSOCIATED WITH A DATA TABLE. VERIFY AND TRY AGAIN")
        sys.exit(1)

        

else: 
    if len(productId.split("-")) !=4:
        print("WARNING: THIS IS LIKELY A NUMBER NOT ASSOCIATED WITH A DATA TABLE. VERIFY AND TRY AGAIN")
        sys.exit(1)

download_tables(str(productId))
df_fullDATA = read_data_compute_df(productId)

cols = list(df_fullDATA.loc[:,'REF_DATE':'UOM'])+ ['SCALAR_FACTOR'] +  ['VALUE']
df_less = df_fullDATA[cols]
df_less2 = df_less.drop(["DGUID"], axis=1)
df_less2 = df_less2.replace({"Aboriginal": "Indigenous"},regex=True)

df_less2  = df_less2.rename(columns={"Aboriginal group": "Indigenous group"})


iteration_nr = df_less2.shape[1]
categories = []
for i in range(iteration_nr-1):
    categories.append(df_less2.iloc[:,i].unique())


all_the_widgets = []
for i in range(len(categories)):
    if i==0:
        a_category = widgets.Dropdown(
                value = categories[i][0],
                options = categories[i], 
                description ='Start Date:', 
                style = style, 
                disabled=False
            )
        b_category = widgets.Dropdown(
                value = categories[i][-1],
                options = categories[i], 
                description ='End Date:', 
                style = style, 
                disabled=False
            )
        all_the_widgets.append(a_category)
        all_the_widgets.append(b_category)
    elif i==1:
        a_category = widgets.Dropdown(
                value = categories[i][0],
                options = categories[i], 
                description ='Location:', 
                style = style, 
                disabled=False
            )
        all_the_widgets.append(a_category)
    elif i==len(categories)-1:
        a_category = widgets.Dropdown(
                value = categories[i][0],
                options = categories[i], 
                description ='Scalar factor:', 
                style = style, 
                disabled=False
            )
        all_the_widgets.append(a_category)

    elif i==len(categories)-2:
        a_category = widgets.Dropdown(
                value = categories[i][0],
                options = categories[i], 
                description ='Units of Measure :', 
                style = style, 
                disabled=False
            )
        all_the_widgets.append(a_category)
    else:
        a_category = widgets.Dropdown(
                value = categories[i][0],
                options = categories[i], 
                description ='Subcategory ' + str(i), 
                style = style, 
                disabled=False
            )
        all_the_widgets.append(a_category)
df_less2.head()

## Select Data Subsets: One-Dimensional Plotting


Use the user menu below to select a cateory within the full subset you are interested in exploring. 

Choose a starting and end date to plot results. 

If there is data available, it will appear under the headers. 

Be careful to select dataframes with actual data in them!. 

Use the Select Dataset button to help you preview the data. 

Press the "Preview Dataset" button to visualize. 

In [ ]:
CD_button = widgets.Button(
    button_style='success',
    description="Preview Dataset", 
    layout=Layout(width='15%', height='30px'),
    style=style
)    
CD_button.on_click( run_4cell )

tab3 = VBox(children=[HBox(children=all_the_widgets[0:3]),
                      HBox(children=all_the_widgets[3:5]),
                      HBox(children=all_the_widgets[5:len(all_the_widgets)]),
                      CD_button])
tab = widgets.Tab(children=[tab3])
tab.set_title(0, 'Load Data Subset')
display(tab)

In [ ]:
df_sub = df_less2[(df_less2["REF_DATE"]>=all_the_widgets[0].value) & 
                  (df_less2["REF_DATE"]<=all_the_widgets[1].value) &
                  (df_less2["GEO"]==all_the_widgets[2].value) &
                  (df_less2["UOM"]==all_the_widgets[-2].value) & 
                  (df_less2["SCALAR_FACTOR"]==all_the_widgets[-1].value) & 
                  (df_less2["Indigenous group"]==all_the_widgets[3].value) &
                  (df_less2["Labour force characteristics"]==all_the_widgets[4].value) & 
                  (df_less2["Educational attainment"]==all_the_widgets[5].value) &
                  (df_less2["Age group"]==all_the_widgets[6].value)]

df_sub.head()

In [ ]:
# TO HANDLE THE REST OF THE COLUMNS, SIMPLY SUBSTITUTE VALUES 
col_name = df_sub.columns[2]

# weather_data = pd.read_csv("DATA.csv",sep=',')
col_name

df_sub_final = df_sub[(df_sub[col_name]==all_the_widgets[3].value)]

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

fig1 = plt.figure(facecolor='w',figsize=(18,18))
plt.subplot(3, 3, 1)
plt.axis('off');


plt.subplot(3, 3, 2)
plt.plot(df_sub_final["REF_DATE"],df_sub_final["VALUE"],'b--',label='Value')
#plt.plot(df_20_USA["REF_DATE"],df_20_USA["VALUE"],'r--',label='U.S. dollar, daily average')
plt.xlabel('Year-Month', fontsize=20)
plt.ylabel('Value',fontsize=20)
plt.title(str(all_the_widgets[3].value) + ", "+  str(all_the_widgets[2].value),fontsize=20)
plt.xticks(rotation=90)
plt.grid(True)


plt.subplot(3, 3, 3);
plt.axis('off');


## Grouping and visualizing data

In the next sections we will group and visualize the data to gather further insights. 

In [ ]:
#load "cufflinks" library under short name "cf"
import cufflinks as cf

#command to display graphics correctly in Jupyter notebook
cf.go_offline()



Let's take a look at how the age groups are distributed by Indigenous group, and let's take a look at the minimum, maximum and average number of people in each category. Units are thousands. Hover over the plot. 

In [ ]:
df_less2.pivot_table(columns= "Indigenous group",index="Age group",values="VALUE",aggfunc=['min','max',"mean"])

#### Table 1. Min, Max and Mean age groups, by Indigenous group.

In [ ]:
df_less2.pivot_table(columns= "Indigenous group",
                     index="Age group",
                     values="VALUE",aggfunc=['mean']).iplot(kind='bar') #values and labels are DataFrame columns

#### Figure 1. Plot average number of people in each age group. 

In [ ]:
df_less2.pivot_table(columns= "Indigenous group",
                     index="Labour force characteristics",
                     values="VALUE",aggfunc=["min",'max','mean'])

#### Table 2. Min, max and mean number of people, by Indigenous group, and their involvement in labour force.

In [ ]:
df_less2[df_less2["Labour force characteristics"]=="Employment rate"].\
pivot_table(columns=["Labour force characteristics","REF_DATE"], \
                     index=["Indigenous group"]).iplot(kind='bar')

#### Figure 2. Average number of people involved in a given aspect of labour force.  

In [ ]:
df_less2.pivot_table(columns= "Indigenous group",
                     index=["Educational attainment","Age group"],
                     values="VALUE",aggfunc=["min",'max','mean'])

#### Table 3. Eductional attainment, by age group and Indigenous group, min, max and average.

In [ ]:
df_less2[df_less2["Educational attainment"]=="Less than high school"].\
pivot_table(columns=["Educational attainment", "REF_DATE"], 
                     index=["Indigenous group","Age group"],aggfunc='mean').iplot(kind='bar')

#### Figure 3. Average number of people, educational attainment level. 

#### Your comments here

Double click this cell to answer the questions:

1. What are the differences in highest educational level by age group? 

2. What are the differences in labour force status, by age group and Indigenous, Metis or First Nations status? 

3. How have employment and unemployment rates have changed since 2007 up until today?

4. How can you use the data in this notebook to help you identify areas of improvement? 

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)